###  Map of countries by emissions

In [2]:
# // 1.  Import packages that we need:
import numpy as np
import pandas as pd
# // Web scraping: 
import requests
import string
from bs4 import BeautifulSoup
# // OS. Sometimes need this for finding working directory:
import os
# // datetime
from datetime import datetime
# // regex library used to detect the presence of particular characters (eg extarcting numbers from string)
import re
from pprint import pprint 

# // package to convert country names and iso codes, eg United States of America -> USA / US
import country_converter as coco
# // for loading JSON data 
import json
import geopandas
import sys

In [3]:
# Step 1: Loop to create list of all countries for 'Electricity net consumption, {}, Annual' Dataset

URL = "https://www.eia.gov/opendata/qb.php?category=2622652"

# // Request the html from the URL:
html = requests.get(URL)

# // Get the soup of this page
soup = BeautifulSoup(html.content, 'html.parser')

In [4]:
# use css selector to extract only relevent links
names = soup.select('div.main_col ul > li')

In [5]:
# 1. find all countries with links to API on eia page
# 2. extract only country name
# 3. then use ISO converter to find character code for each country name
# 4. ready to feed through loop to individually fetch data from each country's api link


countries = []
for i in range(len(soup.select('div.main_col ul > li'))):
    # take text of ith list string 
    to_strip = names[i].text.strip()
    # split on commas as country always contained within
    chunks = to_strip.split(', ')
    # add to master 2nd element to country list
    countries.append(chunks[1])

# create dataframe of results
df_c = pd.DataFrame(countries)

# create new column of ISO 3 codes
# // this has the plus of making it easy to auto-strip out non country data links (eg OECD or World)
df_c['ISO3'] = coco.convert(df_c[0], to='ISO3')


# drop all rows with "not found" for ISO3 code, eg EU / OECD
df_c = df_c[df_c['ISO3'].str.contains("not found")==False]

# drop row for Antarctica (inconsistent and low value data)
df_c = df_c[df_c['ISO3'].str.contains("ATA")==False]

# drops rows with duplicates values in 'Country' or ISO3, duplicate ISO codes for some areas, eg 
#   former Czechoslovakia and Czechia give CZE (although API link for former is different), fortunately in api the ISO3
#    codes will only give data for the current country so effectively cleans results of old time-series
df_c = df_c.drop_duplicates(subset=[0], keep='first')
df_c = df_c.drop_duplicates(subset=['ISO3'], keep='first')

# reset index and drop old index
df_c.reset_index(drop=True, inplace=True)

# Use converter in reverse to give standard format names to all countries ...
#   (happened because converting and dropping duplicates left some mismatched eg: 'South Korea and other Asia' ...
#    was matched with 'KOR' instead of being dropped like OECD was)
df_c[0] = coco.convert(df_c['ISO3'], to='name_short')

df_c.columns = ['Country', 'ISO3']

Africa not found in regex
Asia & Oceania not found in regex
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for Australia and New Zealand
More then one regular expression match for 

In [6]:
## Now build loop to retrieve api results from every country in table
    # build in tidy data format

API_Key = "cKNHOE6oTscEBlfP9TF7LXFAK68fOfbzhTVn3HO0"

# // Set the base url for:
#   'CO2 Emissions, annual, million metric tonnes'
url_base = "http://api.eia.gov/series/?api_key=cKNHOE6oTscEBlfP9TF7LXFAK68fOfbzhTVn3HO0&series_id=INTL.4008-8-{}-MMTCD.A"


# // create list of ISO3 codes (have to add .to_list() otherwise it just returns a pandas series and 'c in codes' wont work)
codes = df_c['ISO3'].to_list()

# ** Kosovo does not have official iso3 code, coco assigned XKX but EIA uses XKS, so manually change otherwise 
#  loop breaks at 'for i in data['series']:' on Kosovo iteration
for i in range(len(codes)):
    if codes[i] == 'XKX':
        codes[i] = 'XKS'

# // create empty df
df = pd.DataFrame()

for c in codes:

    # // Return the index number of the thing we are working with:
    s = codes.index(c)

    # // Build the URL for this iteration of the loop:
    URL = url_base.format(c)

    # // Request the html from the URL:
    data = requests.get(URL).json()

    # // build template array
    base_array = np.array(['Year', 'mtonnes CO2', 'Country'])


    for i in data['series']:
        
        # add all timeseries data to array (format is [Year, <value>])
        array_i = np.array(i['data'])

        # add column with country code in every row
        array_i = np.insert(array_i,array_i.shape[1],c,axis=1)

        # add headers for first result
        if s == 0:
            array_i = np.vstack((base_array, array_i))

        # // Convert array into temporary dataframe:
        df_x = pd.DataFrame(array_i)

        # drops any obersations with NA
        df_x = df_x[df_x[1].str.contains("NA")==False]

        # concat temp df with main df
        df = pd.concat([df, df_x], ignore_index=True)

# set first row as headers and delete row
df.columns = df.iloc[0]
df = df.iloc[1: , :]

In [7]:
df

,Year,mtonnes CO2,Country
1,2019,7.893076967,AFG
2,2018,9.391163087,AFG
3,2017,7.886124149,AFG
4,2016,7.279395964,AFG
5,2015,7.852509211,AFG
...,...,...,...
8747,1984,7.783562489,ZWE
8748,1983,7.584832144,ZWE
8749,1982,7.332897573,ZWE
8750,1981,7.531253431,ZWE


In [8]:
## build dataframe for population

pop_url = "http://api.eia.gov/series/?api_key=cKNHOE6oTscEBlfP9TF7LXFAK68fOfbzhTVn3HO0&series_id=INTL.4702-33-{}-THP.A"


# // create empty df
df_pop = pd.DataFrame()

for c in codes:

    # // Return the index number of the thing we are working with:
    s = codes.index(c)

    # // Build the URL for this iteration of the loop:
    URL = pop_url.format(c)

    # // Request the html from the URL:
    data = requests.get(URL).json()

    # // build template array
    base_array = np.array(['Year', 'Population', 'Country'])


    for i in data['series']:
        
        # add all timeseries data to array (format is [Year, <value>])
        array_i = np.array(i['data'])

        # add column with country code in every row
        array_i = np.insert(array_i,array_i.shape[1],c,axis=1)

        # add headers for first result
        if s == 0:
            array_i = np.vstack((base_array, array_i))

        # // Convert array into temporary dataframe:
        df_x = pd.DataFrame(array_i)

        # drops any obersations with NA
        df_x = df_x[df_x[1].str.contains("NA")==False]

        # concat temp df with main df
        df_pop = pd.concat([df_pop, df_x], ignore_index=True)

In [9]:
# set first row as headers and delete row
df_pop.columns = df_pop.iloc[0]
df_pop = df_pop.iloc[1: , :]

In [10]:
## extract 2019 data from each dataframe

co2_2019 = df[df.Year == '2019']
pop_2019 = df_pop[df_pop.Year == '2019']

In [36]:
## merge datasets

merge = pd.merge(
    co2_2019,
    pop_2019,
    how="inner",
    on="Country",
    left_index=False,
    right_index=False,
    sort=True,
    copy=True,
    indicator=False,
    validate=None,
)

# calculate emissions per capita
# -- pop in thousands, emissions in millions
merge = merge.astype({"mtonnes CO2": float, "Population": float})


merge["CO2 emissions per capita (t)"] = (1000*merge['mtonnes CO2'])/merge['Population']

# // rename columns
merge.columns = ['Year', 'CO2 (million t)', 'ISO3', 'Year', 'Population', 'CO2 emissions per capita (t)']

# drop duplicate column
merge = merge.drop(columns=['Year'])

# round main data for better presentation later
merge['CO2 emissions per capita (t)'] = merge['CO2 emissions per capita (t)'].round(3)

# format pop as actual
merge['Population'] = merge['Population']*1000
merge = merge.astype({"Population": int})

merge['Country'] = coco.convert(merge['ISO3'], to='name_short')


XKS not found in ISO3


In [37]:
# merge with geojson

# import country geojson by Natural Earth, downloaded from https://datahub.io/core/geo-countries#pandas 
geojson = geopandas.read_file('/Users/joshhellings/Documents/OneDrive - University of Bristol/Economics Year 3/Data Science/Project data/globalMap.geojson')

# merge geojson and data
full_dataset = geojson.merge(merge, left_on="ISO_A3", right_on="ISO3")

full_dataset.to_file("/Users/joshhellings/Documents/OneDrive - University of Bristol/Economics Year 3/Data Science/Github Mirror/Project/Data/emissions_perCapita.geojson", driver='GeoJSON')

In [35]:
merge

,CO2 (million t),Country,Population,CO2 emissions per capita (t)
0,1.254453,ABW,106298,11.801
1,7.893077,AFG,38050900,0.207
2,19.361933,AGO,31849800,0.608
3,3.794238,ALB,2879920,1.317
4,276.211962,ARE,9764900,28.286
...,...,...,...,...
197,8.019692,XKS,1794250,4.470
198,10.366728,YEM,29162200,0.355
199,470.358047,ZAF,58650230,8.020
200,6.798094,ZMB,17873850,0.380
